# Training Sub-Image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

%matplotlib widget

## Import next quantum

In [ ]:
with open('unlabeled_data/sources_0.obj', 'rb') as f:
    sources = pickle.load(f)

## Sub-image in widget

In [ ]:
plt.close()
fig, ax = plt.subplots(len(sources), 3, figsize=(10, len(sources)*4))
ax[0][0].set_title('VLASS')
ax[0][1].set_title('VLASS + WISE')
ax[0][2].set_title('WISE')

for i in range(len(sources)):
    ax[i][0].imshow(sources[i].images[0].data, origin='lower', cmap='jet')
    # ax[i][0].set_xlim(0, 128); ax[i][0].set_ylim(0, 128)

    ax[i][1].imshow(sources[i].images[1].data, origin='lower', cmap='cividis')
    ax[i][1].imshow(sources[i].images[0].data, origin='lower', cmap='jet', alpha=0.4)
    # ax[i][1].set_xlim(0, 128); ax[i][1].set_ylim(0, 128)

    ax[i][2].imshow(sources[i].images[1].data, origin='lower', cmap='cividis')
    # ax[i][2].set_xlim(0, 128); ax[i][2].set_ylim(0, 128)

plt.show()

## Save sub-image bounds to fits header and training data file

In [ ]:
for source in sources:
    source.setAllCoords(int(ax[i][1].get_xlim()[0]), int(ax[i][1].get_xlim()[1]), 
        int(ax[i][1].get_ylim()[0]), int(ax[i][1].get_ylim()[1]))

'''for i in range(next_im, next_im+10):
    x_0[i] = int(ax[i][1].get_xlim()[0]); x_1[i] = int(ax[i][1].get_xlim()[1])
    y_0[i] = int(ax[i][1].get_ylim()[0]); y_1[i] = int(ax[i][1].get_ylim()[1])

    vlass[i].header.set('x_0', x_0[i], 'left bound of cutout')
    vlass[i].header.set('x_1', x_1[i], 'right bound of cutout')
    vlass[i].header.set('y_0', y_0[i], 'lower bound of cutout')
    vlass[i].header.set('y_1', y_1[i], 'upper bound of cutout')

    wise[i].header.set('x_0', x_0[i], 'left bound of cutout')
    wise[i].header.set('x_1', x_1[i], 'right bound of cutout')
    wise[i].header.set('y_0', y_0[i], 'lower bound of cutout')
    wise[i].header.set('y_1', y_1[i], 'upper bound of cutout')

    vlass[i].writeto('VLASS_training/VLASS_{}.fits'.format(int(im_name[i])), overwrite=True)
    wise[i].writeto('WISE_training/WISE_{}.fits'.format(int(im_name[i])), overwrite=True)

np.savetxt('training_data.txt', np.array([im_name, x_0, x_1, y_0, y_1]).T, header='im_name x_0 x_1 y_0 y_1', fmt='%i')'''

with open('labeled_data/sources_0.obj', 'wb') as f:
        pickle.dump(sources, f)